# Storage Formats

**Prerequisites**

- [Intro to DataFrames and Series](../p01_pandas_intro/v01_pandas_intro.ipynb)


**Outcomes**

- Understand that data can be saved in various formats  
- Know where to get help on file input and output  
- Know when to use csv, xlsx, feather, and sql formats  


**Data**

- Results for all NFL games between September 1920 to February 2017  

## Outline

- [Storage Formats](#Storage-Formats)  
  - [File Formats](#File-Formats)  
  - [Writing DataFrames](#Writing-DataFrames)  
  - [Reading Files into DataFrames](#Reading-Files-into-DataFrames)  
  - [Practice](#Practice)  

In [ ]:
# Uncomment following line to install on colab
#! pip install qeds

In [ ]:
import pandas as pd
import numpy as np

## File Formats

Data can be saved in a variety of formats

pandas understands how to write and read DataFrames to and from many of
these formats

We defer to the [official
documentation](https://pandas.pydata.org/pandas-docs/stable/io.html)
for a full description of how to interact with all the file formats, but
will briefly discuss a few of them here

## CSV

**What is it?** CSVs store data as plain text (strings) where each row is a
line and columns are separated by `,`

### CSV: Pros

- Widely used (you should be familiar with it)  
- Plain text file (can open on any computer, “future proof”)  
- Can be read from and written to by most data software  

### CSV: Cons

- Not the most efficient way to store or access  
- No formal standard, so there is room for user interpretation on how to
  handle edge cases (e.g. what to do about a data field that itself includes
  a comma)  

### CSV: When to use

- A great default option for most use cases  

## xlsx

**What is it?** xlsx is a binary file format used as Excel’s default.


### xlsx: Pros

- Standard format in many industries  
- Easy to share with colleagues that use Excel  

### xlsx: Cons

- Quite slow to read/write large amounts of data  
- Stores both data and *metadata* like styling and display information
  and even plots. This metadata is not always portable to other file formats
  or programs.  
- Non-human readable

### xlsx: When to use

**When to use**:

- When sharing data with Excel  
- When you would like special formatting to be applied to the
  spreadsheet when viewed in Excel  

## Parquet

**What is it?** Parquet is a custom binary format designed for efficient reading and
writing of data stored in columns.

### Parquet: pros

- *Very* fast  
- Naturally understands all `dtypes` used by pandas, including
  multi-index DataFrames  
- Very common in “big data” systems like Hadoop or Spark  
- Supports various compression algorithms  

### Parquet: Cons

- Binary storage format that is not human-readable  

### Parquet: When to use

- If you have “not small” amounts (> 100 MB) of unchanging data that
  you want to read quickly  
- If you want to store data in an size-and-time-efficient way that may
  be accessed by external systems  

## Feather

**What is it?** Feather is a custom binary format designed for efficient reading and
writing of data stored in columns.

### Feather: Pros

- *Very* fast – even faster than parquet  
- Naturally understands all `dtypes` used by pandas  

### Feather: Cons

- Can only read and write from Python and a handful of other
  programming languages  
- New file format (introduced in March ‘16), so most files don’t come
  in this format  
- Only supports standard pandas index, so you need to `reset_index`
  before saving and then `set_index` after loading  

### Feather: When to use

- Use as an alternative to Parquet if you need the absolute best read and write
  speeds for unchanging datasets  
- Only use when you will not need to access the data in a programming language
  or software outside of Python, R, and Julia  

## SQL

**What is it?** SQL is a language used to interact with relational
databases… [more info](https://en.wikipedia.org/wiki/SQL)

### SQL: Pros

- Well established industry standard for handling data  
- Much of the world’s data is in a SQL database somewhere  

### SQL: Cons

- Complicated: to have full control you need to learn another language
  (SQL)  

### SQL: When to use

- When reading from or writing to existing SQL databases  


**NOTE**: We will likely cover interacting with SQL databases in a later lecture

## JSON

TODO: check what the S means, might just be "script"

**What is is?** JSON is an acronym for "Javascript serialized object notation". It is a very common way to store data, especially when interacting with web services.

Data is stored almost how we would write a Python `dict` by hand using `{` and `}` to provide  key, value pairs

### JSON: Pros

- Very commonly used when interacting with websites and web apis
- Maps naturally into a Python dict
- Human readable
- Can store non-tabular data (each record or row can have different keys)

### JSON: Cons

- Relatively inefficient when stored as an array of records (must repeat column names for every row)
- Non-binary, so can be slow to read and write
- Ambiguous for how to represent certain data (array of records, record of arrays by column, record of arrays by row, etc.)

### JSON: When to use

- When recieving data from or preparing data for web APIs
- When you want to store relatively small amounts of non or semi-structured data

## Writing DataFrames

Let’s now talk about saving a DataFrame to a file.

**Rule of thumb**: To save `df` to a file of type `FOO` use `df.to_FOO`

We'll test this out for a few of the data formats above using some artificial data

**Note**: by default `df2` will be approximately 10 MB. You can change this by adjusting `wanted_mb` in the cell below

In [ ]:
np.random.seed(42)  # makes sure we get the same random numbers each time
df1 = pd.DataFrame(
    np.random.randint(0, 100, size=(10, 4)),
    columns=["a", "b", "c", "d"]
)

wanted_mb = 10  # CHANGE THIS LINE
nrow = 100000
ncol = int(((wanted_mb * 1024**2) / 8) / nrow)
df2 = pd.DataFrame(
    np.random.rand(nrow, ncol),
    columns=["x{}".format(i) for i in range(ncol)]
)

print("df2.shape = ", df2.shape)
print("df2 is approximately {} MB".format(df2.memory_usage().sum() / (1024**2)))

### [df.to_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html)

Let’s start with `df.to_csv`

Without any additional arguments, the `df.to_csv` function will return
a string containing the csv form of the DataFrame:

In [ ]:
# notice the plain text format -- one row per line, columns separated by `'`
print(df1.to_csv())

If we do pass an argument, the first argument will be used as the file name.

In [ ]:
df1.to_csv("df1.csv")

Run the cell below to verify that the file was created.

In [ ]:
import os
os.path.isfile("df1.csv")

Let’s see how long it takes to save `df2` to a file. (Because of the `%%time` at
the top, Jupyter will report the total time to run all code in
the cell)

In [ ]:
%%time
df2.to_csv("df2.csv")

As we will see below, this isn’t as fastest file format we could choose.

### [df.to_excel](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html)

The `df.to_excel` method writes a DataFrame to an excel workbook

The first argument is the name of the file

The second argument is the name of the sheet in that file (this is optional, and is `Sheet1` by default)

In [ ]:
df1.to_excel("df1.xlsx", "df1")

We can write multiple DataFrames to a single workbook, eacn in a different sheet

To do this we use `pd.ExcelWriter(filename)` and then pass the returned object instead of a file name to `df.to_excel`:

In [ ]:
with pd.ExcelWriter("df1.xlsx") as writer:
    df1.to_excel(writer, "df1")
    (df1 + 10).to_excel(writer, "df1 plus 10")

The

```python
with ... as ... :
```

syntax used above is an example of a *context manager*

We don’t need to understand all the details behind what this means
(google it if you are curious)

For our purposes we used it so that Python could:

1. create the `df1.xlsx` file
2. Ensure that the file remains open while pandas writes the two DataFrames
3. Close the file when finished to finalize storing data

<p style="color:red;">

WARNING:

</p>

Saving `df2` to an excel file takes a very long time.

For that reason, we will just show the code and hard-code the output
we saw when we ran the code.

In [ ]:
%%time
df2.to_excel("df2.xlsx")

```python
 Wall time: 25.7 s
```


### [pyarrow.feather.write_feather](https://arrow.apache.org/docs/python/generated/pyarrow.feather.write_feather.html#pyarrow.feather.write_feather)

As noted above, the feather file format was developed for very efficient
reading and writing between Python and your computer

Support for this format is provided by a separate Python package called `pyarrow`

This package is not installed by default

If you do not have it, but would like to install `pyarrow`, uncomment and execute the cell below

In [ ]:
# %pip install pyarrow

The parameters for `pyarrow.feather.write_feather` are the DataFrame and file name

Let’s try it out

In [ ]:
import pyarrow.feather
pyarrow.feather.write_feather(df1, "df1.feather")

In [ ]:
%%time
pyarrow.feather.write_feather(df2, "df2.feather")

An example timing result:

|format|time|
|:---------:|:----------------------:|
|csv|2.66 seconds|
|xlsx|25.7 seconds|
|feather|43 milliseconds|

As you can see, saving this DataFrame in the feather format was far
faster than either CSV or Excel.

## Reading Files into DataFrames

As with the `df.to_FOO` family of methods, there are similar
`pd.read_FOO` functions. (Note: they are in defined pandas, not as
methods on a DataFrame.)

These methods have many more options because data storage can be messy or wrong

We will explore these in more detail in a separate lecture

For now, we just want to highlight the differences in how to read data
from each of the file formats

Let’s start by reading the files we just created to verify that they
match the data we began with

In [ ]:
# notice that index was specified in the first (0th -- why?) column of the file
df1_csv = pd.read_csv("df1.csv", index_col=0)
df1_csv.head()

In [ ]:
df1_xlsx = pd.read_excel("df1.xlsx", "df1", index_col=0)
df1_xlsx.head()

In [ ]:
# notice feather already knows what the index is
df1_feather = pyarrow.feather.read_feather("df1.feather")
df1_feather.head()

`pd.read_FOO`functions can read files stored online

To do this, we pass a URL as the first argument:

In [ ]:
df1_url = "https://storage.googleapis.com/workshop_materials/df1.csv"
df1_web = pd.read_csv(df1_url, index_col=0)
df1_web.head()

## Practice

Now it’s your turn…

In the cell below, the variable `url` contains a web address to a csv
file containing the result of all NFL games from September 1920 to
February 2017

Your task is to do the following:

- Use `pd.read_csv` to read this file into a DataFrame named `nfl`  
- Print the shape and column names of `nfl`  
- Save the DataFrame to a file named `nfl.xlsx`  
- Open the spreadsheet using Excel on your computer  

If you finish quickly, do some basic analysis of the data. Try to do
something interesting. If you get stuck, here are some suggestions for
what to try:

- Compute the average total points in each game (note, you will need to
  sum two of the columns to get total points).  
- Repeat the above calculation, but only for playoff games.  
- Compute the average score for your favorite team (you’ll need to
  consider when they were team1 vs team2).  
- Compute the ratio of “upsets” to total games played. An upset is
  defined as a team with a lower ELO winning the game.  

In [ ]:
url = "https://raw.githubusercontent.com/fivethirtyeight/nfl-elo-game/"
url = url + "3488b7d0b46c5f6583679bc40fb3a42d729abd39/data/nfl_games.csv"

# your code here --- create more cells if necessary

### Cleanup

If you want to remove the files we just created, run the following cell.

In [ ]:
def try_remove(file):
    if os.path.isfile(file):
        os.remove(file)

for df in ["df1", "df2"]:
    for extension in ["csv", "feather", "xlsx"]:
        filename = df + "." + extension
        try_remove(filename)